In [1]:
import os


from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)


from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv())

True

In [2]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")
PINECONE_ENVIRONMENT = os.getenv("PINECONE_ENVIRONMENT")

In [3]:
chat = ChatOpenAI(
    openai_api_key= OPENAI_API_KEY,
    model='gpt-3.5-turbo'
)

In [4]:
messages = [
    SystemMessage(content="""You are a helpful assistant assistants on their queries. Don't mention anything about context, if it is provided in prompt.
                            Also, make sure the students their UID and email address before solving their queries adn save them. You don't need to verify if they are accurate.
                            If you are given that can't be done by you such as sending mails, state that a request has been placed for this to the relevant department"""),
    HumanMessage(content="Hi AI, how are you today?"),
    AIMessage(content="I'm great thank you. How can I help you?"),
]

In [5]:
res = chat(messages)
res

AIMessage(content="I'm an AI assistant, so I don't have feelings in the same way humans do. But I'm here to assist you with any questions or tasks you have. How can I help you today?")

In [6]:
# add latest AI response to messages
messages.append(res)

# now create a new user prompt
prompt = HumanMessage(
    content="Can you help me with any question?"
)
# add to messages
messages.append(prompt)

# send to chat-gpt
res = chat(messages)

print(res.content)

Yes, I can help you with any question you have. Just let me know what you need assistance with, and I'll do my best to provide you with the information or guidance you're looking for.


In [7]:
import pinecone

# get API key from app.pinecone.io and environment from console
pinecone.init(
    api_key=os.getenv('PINECONE_API_KEY'),  
    environment=os.getenv('PINECONE_ENV')  
)

C:\Users\Uday Ganti\anaconda3\lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [8]:
index_list = pinecone.list_indexes()

In [9]:
import time

if PINECONE_INDEX_NAME not in pinecone.list_indexes():
    pinecone.create_index(
        PINECONE_INDEX_NAME,
        dimension=1536,
        metric='cosine'
    )
    # wait for index to finish initialization
    while not pinecone.describe_index(PINECONE_INDEX_NAME).status['ready']:
        time.sleep(1)
        

index = pinecone.Index(PINECONE_INDEX_NAME)

In [10]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.05934,
 'namespaces': {'': {'vector_count': 5934}},
 'total_vector_count': 5934}

In [11]:
from langchain.embeddings.openai import OpenAIEmbeddings

embed_model = OpenAIEmbeddings(model="text-embedding-ada-002")

In [12]:
from langchain.vectorstores import Pinecone

text_field = "text"  # the metadata field that contains our text

# initialize the vector store object
vectorstore = Pinecone(
    index, embed_model.embed_query, text_field
)

C:\Users\Uday Ganti\anaconda3\lib\site-packages\langchain\vectorstores\pinecone.py:59: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


In [13]:
def augment_prompt(query: str):
    # get top 3 results from knowledge base
    results = vectorstore.similarity_search(query, k=3)
    # get the text from the results
    source_knowledge = "\n".join([x.page_content for x in results])
    # feed into an augmented prompt
    augmented_prompt = f"""Using the contexts below, answer the query.

    Contexts:
    {source_knowledge}

    Query: {query}"""
    return augmented_prompt

In [18]:
from langchain.vectorstores import Pinecone

query = "What are the classes thought by pTIM SMITH"

vectorstore.similarity_search(query, k=3)

[Document(page_content="Tim SmithAssistant Professor of Instructionsmith515@usf.eduCampus: St. PetersburgRoom: LPH 420Tim Smith is an assistant professor of instruction in the School of Information Systems and Management on USF's St. Petersburg campus. He came to USF in 2022 from Iowa State University, where he was an assistant teaching professor in the Ivy College of Business.Smith's research interests lie in organizational routines, technology adoption processes, business analytics, and enterprise systems development. His work has been published in theJournal of Information Systems Education, theTeaching Mathematics and Computer Sciencejournal, and currently serves as the technical editor for theJournal of Information Systems Education. He has presented research at the Hawaii International Conference on System Sciences, Americas Conference on Information Systems, and International Conference on Information systems. He has also won several awards, including the best paper award at the

In [19]:
print(augment_prompt(query))

Using the contexts below, answer the query.

    Contexts:
    Tim SmithAssistant Professor of Instructionsmith515@usf.eduCampus: St. PetersburgRoom: LPH 420Tim Smith is an assistant professor of instruction in the School of Information Systems and Management on USF's St. Petersburg campus. He came to USF in 2022 from Iowa State University, where he was an assistant teaching professor in the Ivy College of Business.Smith's research interests lie in organizational routines, technology adoption processes, business analytics, and enterprise systems development. His work has been published in theJournal of Information Systems Education, theTeaching Mathematics and Computer Sciencejournal, and currently serves as the technical editor for theJournal of Information Systems Education. He has presented research at the Hawaii International Conference on System Sciences, Americas Conference on Information Systems, and International Conference on Information systems. He has also won several awards

In [20]:
# create a new user prompt
prompt = HumanMessage(
    content=augment_prompt(query)
)
# add to messages
messages.append(prompt)

res = chat(messages)

print(res.content)

Tim Smith, an Assistant Professor of Instruction, teaches the following classes at USF:

1. ACG 3113 – Intermediate Financial Accounting II
2. ACG 4123 – Intermediate Financial Accounting III
3. ACG 7936 – Seminar in Archival Accounting Research

Please note that these classes are related to accounting and not specifically mentioned as being taught by Tim Smith. If you are looking for classes taught by Tim Smith in a different department, please provide more information.


In [17]:
# now create a new user prompt
prompt = HumanMessage(
    content="I have sent my application 3 days ago and not have not recieved any feedback email about it."
)
# add to messages
messages.append(prompt)

# send to chat-gpt
res = chat(messages)

print(res.content)

messages.append(res)

I apologize for the inconvenience. As an AI assistant, I don't have access to personal information or specific application processes. However, it's possible that the application process may take some time, and it's not unusual to not receive immediate feedback. I recommend reaching out to the relevant department or contact person directly to inquire about the status of your application. They will be able to provide you with the most accurate information.
